In [2]:
"""
Hierarchical Document Splitter for Technical Documentation
계층적 기술 문서 분할기

특징:
1. 출처 URL 기반 1차 분할 (각 페이지별 독립성 보장)
2. 헤더 기반 계층적 분할 (# → ## → ### → ####)
3. 테이블과 코드 블록의 구조적 보존
4. 섹션별 컨텍스트 메타데이터 강화
5. 전문용어 추출 및 태깅
"""

import re
from typing import List, Dict, Tuple
from dataclasses import dataclass
from langchain.docstore.document import Document


@dataclass
class DocumentSection:
    """문서 섹션 정보"""
    content: str
    header_path: List[str]  # ["h1", "h2", "h3", "h4"] 계층
    level: int  # 헤더 레벨 (1-4)
    source_url: str
    section_id: str
    start_pos: int
    end_pos: int


class TechnicalTermExtractor:
    """기술 문서 전용 용어 추출기"""
    
    def __init__(self):
        self.patterns = {
            'api_codes': re.compile(r'\b[A-Z]{2,}[A-Z0-9_]*\b'),  # API, SDK, PNS 등
            'camel_case': re.compile(r'\b[a-z]+[A-Z][a-zA-Z0-9]*\b'),  # purchaseState 등
            'snake_case': re.compile(r'\b[a-z]+_[a-z0-9_]+\b'),  # query_purchases 등
            'version_codes': re.compile(r'\b[Vv]\d+(\.\d+)*\b'),  # V21, v1.0 등
            'http_codes': re.compile(r'\b[1-5]\d{2}\b'),  # 200, 404 등
            'method_names': re.compile(r'\b\w+\(\)\b'),  # launchPurchaseFlow() 등
            'class_names': re.compile(r'\b[A-Z][a-zA-Z0-9]*(?:Client|Listener|Params|Data|Result)\b'),
            'korean_tech': re.compile(r'(?:인앱|결제|구매|구독|월정액|상품|토큰|라이선스)'),
        }
        
        # 제외할 일반적인 단어들 (노이즈 감소)
        self.exclude_terms = {
            'IF', 'OR', 'TO', 'IS', 'ON', 'IN', 'AT', 'BY', 'UP', 'NO', 'OK',
            'GET', 'SET', 'PUT', 'POST', 'NEW', 'OLD', 'MAX', 'MIN', 'END'
        }
    
    def extract_terms(self, text: str) -> Dict[str, List[str]]:
        """텍스트에서 기술 용어들을 카테고리별로 추출"""
        terms = {}
        for category, pattern in self.patterns.items():
            matches = pattern.findall(text)
            # 중복 제거 및 제외 단어 필터링
            unique_matches = list(set(match for match in matches 
                                   if match.upper() not in self.exclude_terms))
            if unique_matches:
                terms[category] = unique_matches
        return terms


class HierarchicalSplitter:
    """계층적 문서 분할기"""
    
    def __init__(self, 
                 chunk_size: int = 1000,
                 overlap_ratio: float = 0.1,
                 preserve_tables: bool = True,
                 preserve_code: bool = True):
        self.chunk_size = chunk_size
        self.overlap_size = int(chunk_size * overlap_ratio)
        self.preserve_tables = preserve_tables
        self.preserve_code = preserve_code
        self.term_extractor = TechnicalTermExtractor()
        
        # 출처 URL 패턴
        self.source_pattern = re.compile(r'^출처:\s*(https?://[^\s]+)', re.MULTILINE)
        
        # 헤더 패턴 (마크다운)
        self.header_patterns = {
            1: re.compile(r'^#\s+(.+)$', re.MULTILINE),
            2: re.compile(r'^##\s+(.+)$', re.MULTILINE),
            3: re.compile(r'^###\s+(.+)$', re.MULTILINE),
            4: re.compile(r'^####\s+(.+)$', re.MULTILINE),
        }
        
        # 테이블 패턴
        self.table_pattern = re.compile(r'^\|.+\|$', re.MULTILINE)
        
        # 코드 블록 패턴
        self.code_pattern = re.compile(r'```(\w+)?\n(.*?)\n```', re.DOTALL)
    
    def split_by_source(self, content: str) -> List[Tuple[str, str]]:
        """출처 URL 기준으로 문서를 1차 분할"""
        sources = list(self.source_pattern.finditer(content))
        sections = []
        
        for i, source_match in enumerate(sources):
            start_pos = source_match.start()
            end_pos = sources[i + 1].start() if i + 1 < len(sources) else len(content)
            
            source_url = source_match.group(1)
            section_content = content[start_pos:end_pos].strip()
            sections.append((source_url, section_content))
        
        return sections
    
    def extract_headers(self, content: str) -> List[Dict]:
        """텍스트에서 헤더 정보 추출"""
        headers = []
        
        for level in range(1, 5):  # h1 ~ h4
            pattern = self.header_patterns[level]
            for match in pattern.finditer(content):
                headers.append({
                    'level': level,
                    'title': match.group(1).strip(),
                    'position': match.start(),
                    'end_position': match.end()
                })
        
        # 위치 순으로 정렬
        headers.sort(key=lambda x: x['position'])
        return headers
    
    def build_hierarchy_path(self, headers: List[Dict], current_pos: int) -> List[str]:
        """현재 위치까지의 헤더 계층 경로 구성"""
        path = [''] * 4  # h1, h2, h3, h4
        
        for header in headers:
            if header['position'] > current_pos:
                break
            level = header['level']
            path[level - 1] = header['title']
            # 하위 레벨 초기화
            for i in range(level, 4):
                if i > level - 1:
                    path[i] = ''
        
        # 빈 문자열 제거하고 실제 경로만 반환
        return [p for p in path if p]
    
    def extract_tables(self, content: str) -> List[Dict]:
        """테이블 구조 추출 및 파싱"""
        tables = []
        lines = content.split('\n')
        table_start = None
        table_rows = []
        
        for i, line in enumerate(lines):
            if self.table_pattern.match(line):
                if table_start is None:
                    table_start = i
                table_rows.append(line)
            else:
                if table_start is not None:
                    # 테이블 종료
                    if len(table_rows) >= 2:  # 최소 헤더 + 데이터 1행
                        tables.append({
                            'start_line': table_start,
                            'end_line': i,
                            'rows': table_rows,
                            'content': '\n'.join(table_rows)
                        })
                    table_start = None
                    table_rows = []
        
        # 마지막 테이블 처리
        if table_start is not None and len(table_rows) >= 2:
            tables.append({
                'start_line': table_start,
                'end_line': len(lines),
                'rows': table_rows,
                'content': '\n'.join(table_rows)
            })
        
        return tables
    
    def extract_code_blocks(self, content: str) -> List[Dict]:
        """코드 블록 추출"""
        blocks = []
        for match in self.code_pattern.finditer(content):
            language = match.group(1) or 'text'
            code_content = match.group(2)
            blocks.append({
                'language': language,
                'content': code_content,
                'full_match': match.group(0),
                'start_pos': match.start(),
                'end_pos': match.end()
            })
        return blocks
    
    def split_content_by_headers(self, content: str, source_url: str) -> List[DocumentSection]:
        """헤더 기준으로 내용을 계층적으로 분할"""
        headers = self.extract_headers(content)
        sections = []
        
        if not headers:
            # 헤더가 없는 경우 전체를 하나의 섹션으로
            section = DocumentSection(
                content=content,
                header_path=[],
                level=0,
                source_url=source_url,
                section_id=f"section_0",
                start_pos=0,
                end_pos=len(content)
            )
            sections.append(section)
            return sections
        
        # 첫 번째 헤더 이전 내용
        if headers[0]['position'] > 0:
            intro_content = content[:headers[0]['position']].strip()
            if intro_content:
                section = DocumentSection(
                    content=intro_content,
                    header_path=[],
                    level=0,
                    source_url=source_url,
                    section_id=f"section_intro",
                    start_pos=0,
                    end_pos=headers[0]['position']
                )
                sections.append(section)
        
        # 헤더별 섹션 분할
        for i, header in enumerate(headers):
            start_pos = header['end_position']
            end_pos = headers[i + 1]['position'] if i + 1 < len(headers) else len(content)
            
            section_content = content[start_pos:end_pos].strip()
            if not section_content:
                continue
                
            hierarchy_path = self.build_hierarchy_path(headers, header['position'])
            
            section = DocumentSection(
                content=f"# {header['title']}\n\n{section_content}",
                header_path=hierarchy_path,
                level=header['level'],
                source_url=source_url,
                section_id=f"section_{i}",
                start_pos=start_pos,
                end_pos=end_pos
            )
            sections.append(section)
        
        return sections
    
    def chunk_large_section(self, section: DocumentSection) -> List[DocumentSection]:
        """큰 섹션을 청크로 분할"""
        if len(section.content) <= self.chunk_size:
            return [section]
        
        chunks = []
        content = section.content
        start_idx = 0
        chunk_id = 0
        
        while start_idx < len(content):
            end_idx = start_idx + self.chunk_size
            
            # 자연스러운 분할점 찾기 (문단, 문장 경계)
            if end_idx < len(content):
                # 문단 경계 찾기
                para_break = content.rfind('\n\n', start_idx, end_idx)
                if para_break > start_idx:
                    end_idx = para_break + 2
                else:
                    # 문장 경계 찾기
                    sent_break = content.rfind('. ', start_idx, end_idx)
                    if sent_break > start_idx:
                        end_idx = sent_break + 2
            
            chunk_content = content[start_idx:end_idx]
            
            chunk_section = DocumentSection(
                content=chunk_content,
                header_path=section.header_path,
                level=section.level,
                source_url=section.source_url,
                section_id=f"{section.section_id}_chunk_{chunk_id}",
                start_pos=section.start_pos + start_idx,
                end_pos=section.start_pos + end_idx
            )
            chunks.append(chunk_section)
            
            # 다음 청크 시작점 (오버랩 적용)
            start_idx = max(start_idx + 1, end_idx - self.overlap_size)
            chunk_id += 1
        
        return chunks
    
    def create_document_with_metadata(self, section: DocumentSection) -> Document:
        """DocumentSection을 LangChain Document로 변환 (메타데이터 포함)"""
        
        # 기술 용어 추출
        tech_terms = self.term_extractor.extract_terms(section.content)
        
        # 테이블 및 코드 블록 추출
        tables = self.extract_tables(section.content)
        code_blocks = self.extract_code_blocks(section.content)
        
        # 컨텐츠 타입 결정
        content_types = []
        if tables:
            content_types.append('table')
        if code_blocks:
            content_types.append('code')
        if not content_types:
            content_types.append('text')
        
        # 메타데이터 구성
        metadata = {
            'source_url': section.source_url,
            'header_path': section.header_path,
            'section_hierarchy': ' > '.join(section.header_path),
            'header_level': section.level,
            'section_id': section.section_id,
            'content_types': content_types,
            'technical_terms': tech_terms,
            'has_tables': len(tables) > 0,
            'has_code': len(code_blocks) > 0,
            'code_languages': [cb['language'] for cb in code_blocks],
            'table_count': len(tables),
            'code_block_count': len(code_blocks),
            'content_length': len(section.content),
            'start_pos': section.start_pos,
            'end_pos': section.end_pos
        }
        
        # 헤더 경로를 콘텐츠에 포함 (검색 개선)
        content_with_context = section.content
        if section.header_path:
            context_header = f"[{' > '.join(section.header_path)}]\n\n"
            content_with_context = context_header + content_with_context
        
        return Document(
            page_content=content_with_context,
            metadata=metadata
        )
    
    def split_document(self, content: str) -> List[Document]:
        """메인 분할 메소드"""
        documents = []
        
        # 1단계: 출처 URL 기준으로 분할
        source_sections = self.split_by_source(content)
        
        for source_url, section_content in source_sections:
            # 2단계: 헤더 기준으로 계층적 분할
            doc_sections = self.split_content_by_headers(section_content, source_url)
            
            for section in doc_sections:
                # 3단계: 큰 섹션은 청크로 분할
                chunks = self.chunk_large_section(section)
                
                # 4단계: Document 객체 생성
                for chunk in chunks:
                    doc = self.create_document_with_metadata(chunk)
                    documents.append(doc)
        
        return documents



In [3]:
## ONLY TEST ##

# 사용 예제 및 테스트 함수
def test_splitter():
    """분할기 테스트"""
    
    # 샘플 텍스트 (실제 문서 구조 모방)
    sample_content = """
출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
# 원스토어 인앱결제 API V7(SDK V21) 연동 안내

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.

## 개요

원스토어 인앱결제(IAP)는 안드로이드 앱 내에서 상품을 판매하는 서비스입니다.

### PurchaseClient 초기화

```kotlin
val purchaseClient = PurchaseClient.getClient(activity)
```

### 상품 구매 요청

launchPurchaseFlow() 메소드를 사용합니다.

| 파라미터 | 타입 | 설명 |
|---------|-----|-----|
| productId | String | 상품 ID |
| purchaseParams | PurchaseParams | 구매 파라미터 |

출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21/pns.md
# PNS(Payment Notification Service) 이용하기

PNS는 결제 상태를 서버로 전송하는 서비스입니다.

## PNS 설정

개발사 서버에서 API를 구현해야 합니다.
"""
    
    splitter = HierarchicalSplitter(chunk_size=500)
    documents = splitter.split_document(sample_content)
    
    print(f"총 {len(documents)}개의 문서로 분할됨\n")
    
    for i, doc in enumerate(documents):
        print(f"=== 문서 {i+1} ===")
        print(f"내용: {doc.page_content[:100]}...")
        print(f"출처: {doc.metadata['source_url']}")
        print(f"계층: {doc.metadata['section_hierarchy']}")
        print(f"타입: {doc.metadata['content_types']}")
        if doc.metadata['technical_terms']:
            print(f"기술용어: {doc.metadata['technical_terms']}")
        print()


# if __name__ == "__main__":
#     test_splitter()

test_splitter()

총 8개의 문서로 분할됨

=== 문서 1 ===
내용: 출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md...
출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
계층: 
타입: ['text']
기술용어: {'version_codes': ['']}

=== 문서 2 ===
내용: [원스토어 인앱결제 API V7(SDK V21) 연동 안내]

# 원스토어 인앱결제 API V7(SDK V21) 연동 안내

원스토어의 최신 인앱결제 API V7(SDK V21)이...
출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
계층: 원스토어 인앱결제 API V7(SDK V21) 연동 안내
타입: ['text']
기술용어: {'api_codes': ['API', 'SDK'], 'version_codes': [''], 'korean_tech': ['인앱', '결제']}

=== 문서 3 ===
내용: [원스토어 인앱결제 API V7(SDK V21) 연동 안내 > 개요]

# 개요

원스토어 인앱결제(IAP)는 안드로이드 앱 내에서 상품을 판매하는 서비스입니다....
출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
계층: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 > 개요
타입: ['text']
기술용어: {'api_codes': ['IAP'], 'korean_tech': ['인앱', '상품', '결제']}

=== 문서 4 ===
내용: [원스토어 인앱결제 API V7(SDK V21) 연동 안내 > 개요 > PurchaseClient 초기화]

# PurchaseClient 초기화

```kotlin
val pur...
출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
계

In [50]:
"""
Context-Aware Retriever for Technical Documentation
컨텍스트 인식 기술문서 검색기

특징:
1. 전문용어의 맥락적 의미 고려
2. 섹션별 컨텍스트 보존
3. 다단계 리랭킹 (관련성, 맥락, 기술정확성)
4. 쿼리 확장 (동의어, 약어)
5. 노이즈 필터링 (중요도 기반)
"""

import re
from typing import List, Dict, Any, Tuple, Set
from dataclasses import dataclass
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever
from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun


@dataclass
class SearchContext:
    """검색 컨텍스트 정보"""
    query: str
    query_terms: List[str]
    tech_terms: Dict[str, List[str]]
    query_type: str  # 'how_to', 'what_is', 'api_spec', 'error_debug'
    context_keywords: Set[str]


class QueryAnalyzer:
    """쿼리 분석 및 확장"""
    
    def __init__(self):
        # 기술 용어 동의어 사전
        self.tech_synonyms = {
            'IAP': ['인앱결제', '인앱', '결제'],
            'SDK': ['소프트웨어개발키트', '개발킷'],
            'API': ['인터페이스', '연동'],
            'PNS': ['결제알림서비스', 'Payment Notification Service'],
            'PurchaseClient': ['구매클라이언트', '결제클라이언트'],
            'purchaseState': ['구매상태', '결제상태'],
            'purchaseToken': ['구매토큰', '결제토큰'],
            'acknowledge': ['구매확인', '승인'],
            'consume': ['소비', '사용완료'],
            'subscription': ['구독', '정기결제'],
            'recurring': ['월정액', '자동결제', '반복결제']
        }
        
        # 쿼리 타입 패턴
        self.query_patterns = {
            'how_to': re.compile(r'(?:어떻게|방법|사용|구현|적용|설정|연동)'),
            'what_is': re.compile(r'(?:무엇|뭔가요|란|이란|설명|개념)'),
            'api_spec': re.compile(r'(?:파라미터|리턴|응답|요청|스펙|명세)'),
            'error_debug': re.compile(r'(?:에러|오류|문제|실패|안됨|해결)')
        }
        
        # 한국어 조사 패턴
        self.particle_pattern = re.compile(r'([가-힣]+)(?:은|는|이|가|을|를|의|에|와|과|도|로|으로|부터|까지|만)')
    
    def expand_query(self, query: str) -> List[str]:
        """쿼리 확장 (동의어, 약어)"""
        expanded = [query]
        
        for term, synonyms in self.tech_synonyms.items():
            if term.lower() in query.lower():
                for synonym in synonyms:
                    expanded.append(query.replace(term, synonym))
            else:
                for synonym in synonyms:
                    if synonym in query:
                        expanded.append(query.replace(synonym, term))
        
        return list(set(expanded))
    
    def extract_query_terms(self, query: str) -> List[str]:
        """쿼리에서 핵심 용어 추출"""
        # 조사 제거
        query_cleaned = self.particle_pattern.sub(r'\1', query)
        
        terms = []
        
        # 기술 용어 추출
        for term in self.tech_synonyms.keys():
            if term.lower() in query_cleaned.lower():
                terms.append(term)
        
        # 영어 용어 추출
        english_terms = re.findall(r'\b[A-Za-z][A-Za-z0-9_]*\b', query_cleaned)
        terms.extend([t for t in english_terms if len(t) > 2])
        
        # 한글 명사 추출 (간단한 휴리스틱)
        korean_terms = re.findall(r'[가-힣]{2,}', query_cleaned)
        terms.extend(korean_terms)
        
        return list(set(terms))
    
    def classify_query_type(self, query: str) -> str:
        """쿼리 타입 분류"""
        for query_type, pattern in self.query_patterns.items():
            if pattern.search(query):
                return query_type
        return 'general'
    
    def analyze_query(self, query: str) -> SearchContext:
        """쿼리 종합 분석"""
        terms = self.extract_query_terms(query)
        query_type = self.classify_query_type(query)
        
        # 기술 용어 분류
        tech_terms: Dict[str, List[str]] = {
            'api_terms': [],
            'method_terms': [],
            'concept_terms': []
        }
        
        for term in terms:
            if term in self.tech_synonyms or term.upper() in self.tech_synonyms:
                tech_terms['api_terms'].append(term)
            elif '()' in term or 'Client' in term or 'Listener' in term:
                tech_terms['method_terms'].append(term)
            else:
                tech_terms['concept_terms'].append(term)
        
        context_keywords = set(terms)
        
        return SearchContext(
            query=query,
            query_terms=terms,
            tech_terms=tech_terms,
            query_type=query_type,
            context_keywords=context_keywords
        )


class ContextScorer:
    """문맥 기반 점수 계산"""
    
    def __init__(self):
        # 섹션 타입별 가중치
        self.section_weights = {
            'api_spec': {'how_to': 0.8, 'what_is': 0.6, 'api_spec': 1.0, 'error_debug': 0.7},
            'code': {'how_to': 1.0, 'what_is': 0.5, 'api_spec': 0.8, 'error_debug': 0.9},
            'table': {'how_to': 0.7, 'what_is': 0.8, 'api_spec': 1.0, 'error_debug': 0.6},
            'text': {'how_to': 0.9, 'what_is': 1.0, 'api_spec': 0.7, 'error_debug': 0.8}
        }
        
        # 기술 용어 가중치
        self.term_weights = {
            'api_terms': 1.0,
            'method_terms': 0.9,
            'concept_terms': 0.7
        }
    
    def calculate_term_relevance(self, doc: Document, search_context: SearchContext) -> float:
        """용어 관련성 점수"""
        content = doc.page_content.lower()
        metadata = doc.metadata
        
        score = 0.0
        total_weight = 0.0
        
        # 쿼리 용어 매칭
        for term in search_context.query_terms:
            if term.lower() in content:
                score += 1.0
                total_weight += 1.0
        
        # 기술 용어 매칭 (가중치 적용)
        for term_type, terms in search_context.tech_terms.items():
            weight = self.term_weights.get(term_type, 0.5)
            for term in terms:
                if term.lower() in content:
                    score += weight
                    total_weight += weight
        
        # 메타데이터의 기술 용어 매칭
        doc_tech_terms = metadata.get('technical_terms', {})
        for term_type, terms in doc_tech_terms.items():
            for term in terms:
                if term.lower() in search_context.query.lower():
                    score += 0.5
                    total_weight += 0.5
        
        return score / max(total_weight, 1.0)
    
    def calculate_context_relevance(self, doc: Document, search_context: SearchContext) -> float:
        """문맥 관련성 점수"""
        metadata = doc.metadata
        content_types = metadata.get('content_types', ['text'])
        query_type = search_context.query_type
        
        # 컨텐츠 타입과 쿼리 타입 매칭
        type_score = 0.0
        for content_type in content_types:
            if content_type in self.section_weights:
                type_score += self.section_weights[content_type].get(query_type, 0.5)
        
        type_score = type_score / len(content_types)
        
        # 섹션 계층 관련성
        hierarchy = metadata.get('section_hierarchy', '')
        hierarchy_score = 0.0
        
        for keyword in search_context.context_keywords:
            if keyword.lower() in hierarchy.lower():
                hierarchy_score += 0.2
        
        hierarchy_score = min(hierarchy_score, 1.0)
        
        return (type_score + hierarchy_score) / 2.0
    
    def calculate_technical_accuracy(self, doc: Document, search_context: SearchContext) -> float:
        """기술적 정확성 점수"""
        metadata = doc.metadata
        content = doc.page_content.lower()
        
        # 코드 블록과 테이블의 가중치
        accuracy_score = 0.5  # 기본 점수
        
        if metadata.get('has_code'):
            accuracy_score += 0.2
        
        if metadata.get('has_tables'):
            accuracy_score += 0.15
        
        # API 관련 키워드 밀도
        api_keywords = ['api', 'sdk', 'client', 'params', 'response', 'request']
        keyword_count = sum(1 for kw in api_keywords if kw in content)
        accuracy_score += min(keyword_count * 0.05, 0.25)
        
        return min(accuracy_score, 1.0)


class NoiseFilter:
    """노이즈 필터링"""
    
    def __init__(self):
        # 일반적인 노이즈 용어들
        self.noise_terms = {
            'common_words': {'the', 'and', 'or', 'is', 'are', 'was', 'were', 'for', 'to', 'in', 'on'},
            'filler_korean': {'입니다', '있습니다', '합니다', '때문에', '그리고', '하지만', '또한'},
            'generic_tech': {'data', 'info', 'value', 'result', 'success', 'error'}
        }
    
    def calculate_noise_score(self, doc: Document, search_context: SearchContext) -> float:
        """노이즈 점수 계산 (낮을수록 좋음)"""
        content = doc.page_content.lower()
        content_length = len(content)
        
        if content_length == 0:
            return 1.0
        
        noise_count = 0
        total_terms = len(content.split())
        
        # 일반적인 노이즈 용어 카운트
        for noise_category, terms in self.noise_terms.items():
            for term in terms:
                noise_count += content.count(term.lower())
        
        # 반복되는 문구 패널티
        sentences = content.split('.')
        unique_sentences = set(sentences)
        repetition_penalty = 1 - (len(unique_sentences) / max(len(sentences), 1))
        
        # 노이즈 비율 계산
        noise_ratio = noise_count / max(total_terms, 1)
        total_noise_score = min((noise_ratio + repetition_penalty) / 2, 1.0)
        
        return total_noise_score
    
    def filter_by_relevance_threshold(self, docs: List[Document], 
                                    scores: List[float], 
                                    threshold: float = 0.3) -> Tuple[List[Document], List[float]]:
        """관련성 임계값으로 필터링"""
        filtered_docs = []
        filtered_scores = []
        
        for doc, score in zip(docs, scores):
            if score >= threshold:
                filtered_docs.append(doc)
                filtered_scores.append(score)
        
        return filtered_docs, filtered_scores


class ContextAwareRetriever(BaseRetriever):
    """컨텍스트 인식 검색기"""
    
    def __init__(self, 
                 base_retriever: BaseRetriever,
                 rerank_top_k: int = 20,
                 final_top_k: int = 5,
                 enable_query_expansion: bool = True,
                 noise_threshold: float = 0.7):
        
        self.base_retriever = base_retriever
        self.rerank_top_k = rerank_top_k
        self.final_top_k = final_top_k
        self.enable_query_expansion = enable_query_expansion
        self.noise_threshold = noise_threshold
        
        self.query_analyzer = QueryAnalyzer()
        self.context_scorer = ContextScorer()
        self.noise_filter = NoiseFilter()
    
    def _get_relevant_documents(self, 
                              query: str, 
                              *, 
                              run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        """메인 검색 메소드"""
        
        # 1. 쿼리 분석
        search_context = self.query_analyzer.analyze_query(query)
        
        # 2. 쿼리 확장 (옵션)
        queries_to_search = [query]
        if self.enable_query_expansion:
            expanded_queries = self.query_analyzer.expand_query(query)
            queries_to_search.extend(expanded_queries[:3])  # 최대 3개 확장
        
        # 3. 기본 검색 수행
        all_docs = []
        for q in queries_to_search:
            docs = self.base_retriever.get_relevant_documents(q, run_manager=run_manager)
            all_docs.extend(docs)
        
        # 중복 제거 (page_content 기준)
        unique_docs = []
        seen_contents = set()
        for doc in all_docs:
            content_hash = hash(doc.page_content)
            if content_hash not in seen_contents:
                unique_docs.append(doc)
                seen_contents.add(content_hash)
        
        # 상위 K개만 리랭킹 대상으로
        docs_to_rerank = unique_docs[:self.rerank_top_k]
        
        # 4. 다단계 리랭킹
        reranked_docs = self._rerank_documents(docs_to_rerank, search_context)
        
        # 5. 최종 결과 반환
        return reranked_docs[:self.final_top_k]
    
    def _rerank_documents(self, docs: List[Document], search_context: SearchContext) -> List[Document]:
        """다단계 리랭킹"""
        
        scored_docs = []
        
        for doc in docs:
            # 점수 계산
            term_score = self.context_scorer.calculate_term_relevance(doc, search_context)
            context_score = self.context_scorer.calculate_context_relevance(doc, search_context)
            accuracy_score = self.context_scorer.calculate_technical_accuracy(doc, search_context)
            noise_score = self.noise_filter.calculate_noise_score(doc, search_context)
            
            # 종합 점수 (노이즈는 패널티)
            final_score = (term_score * 0.4 + 
                          context_score * 0.3 + 
                          accuracy_score * 0.2 + 
                          (1 - noise_score) * 0.1)
            
            scored_docs.append((doc, final_score))
        
        # 점수순 정렬
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        # 노이즈 임계값으로 필터링
        filtered_docs = []
        for doc, score in scored_docs:
            noise_score = self.noise_filter.calculate_noise_score(doc, search_context)
            if noise_score <= self.noise_threshold:
                filtered_docs.append(doc)
        
        return filtered_docs if filtered_docs else [doc for doc, _ in scored_docs]
    
    def get_search_analytics(self, query: str) -> Dict[str, Any]:
        """검색 분석 정보 반환 (디버깅용)"""
        search_context = self.query_analyzer.analyze_query(query)
        
        return {
            'query_terms': search_context.query_terms,
            'query_type': search_context.query_type,
            'tech_terms': search_context.tech_terms,
            'context_keywords': list(search_context.context_keywords)
        }


# 사용 예제
def create_context_aware_retriever_example():
    """컨텍스트 인식 검색기 생성 예제"""
    
    # 가상의 기본 검색기 (실제로는 FAISS, BM25 등을 사용)
    class MockRetriever(BaseRetriever):
        def __init__(self, documents):
            self.documents = documents
        
        def _get_relevant_documents(self, query, *, run_manager):
            # 간단한 키워드 매칭 검색
            results = []
            for doc in self.documents:
                if any(term.lower() in doc.page_content.lower() 
                      for term in query.split()):
                    results.append(doc)
            return results[:10]
    
    # 샘플 문서들
    sample_docs = [
        Document(
            page_content="PurchaseClient를 사용하여 인앱결제를 구현하는 방법",
            metadata={'content_types': ['text'], 'technical_terms': {'api_terms': ['PurchaseClient']}}
        ),
        Document(
            page_content="purchaseState 값으로 구매 상태를 확인할 수 있습니다",
            metadata={'content_types': ['text'], 'technical_terms': {'method_terms': ['purchaseState']}}
        )
    ]
    
    base_retriever = MockRetriever(sample_docs)
    context_retriever = ContextAwareRetriever(base_retriever)
    
    return context_retriever


# 테스트 함수
def test_retriever():
    """검색기 테스트"""
    retriever = create_context_aware_retriever_example()
    
    queries = [
        "PurchaseClient 사용법이 뭔가요?",
        "purchaseState 값은 무엇인가요?",
        "인앱결제 에러 해결 방법"
    ]
    
    for query in queries:
        print(f"쿼리: {query}")
        analytics = retriever.get_search_analytics(query)
        print(f"분석: {analytics}")
        
        # results = retriever.get_relevant_documents(query)
        # print(f"결과: {len(results)}개 문서")
        print("-" * 50)


# if __name__ == "__main__":
#     test_retriever()


In [51]:
"""
Optimized RAG Pipeline for Technical Documentation
기술문서 최적화 RAG 파이프라인

특징:
1. Ollama 기반 환경 (bge-m3, exaone3.5) 지원
2. 계층적 문서 분할 + 컨텍스트 인식 검색
3. 하이브리드 검색 (FAISS + BM25 + 메타데이터 필터링)
4. 다단계 리랭킹 및 노이즈 필터링
5. 스트리밍 응답 지원
6. 검색 분석 및 디버깅 기능
"""

import os
import json
import time
from typing import List, Dict, Any, Optional, Union

# LangChain imports
from langchain.docstore.document import Document
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# # 로컬 모듈 imports
# from hierarchical_splitter import HierarchicalSplitter
# from context_aware_retriever import ContextAwareRetriever


class OptimizedRAGPipeline:
    """최적화된 RAG 파이프라인"""
    
    def __init__(self,
                 embed_model: str = "bge-m3:latest",
                 llm_model: str = "exaone3.5:latest",
                 data_file: str = "data/dev_center_guide_allmd_touched.md",
                 vector_store_path: str = "models/faiss_optimized",
                 chunk_size: int = 1000,
                 overlap_ratio: float = 0.1,
                 final_top_k: int = 5):
        
        # 모델 설정
        self.embed_model = embed_model
        self.llm_model = llm_model
        self.data_file = data_file
        self.vector_store_path = vector_store_path
        
        # 분할 설정
        self.chunk_size = chunk_size
        self.overlap_ratio = overlap_ratio
        self.final_top_k = final_top_k
        
        # 구성요소 초기화
        self.embeddings: Optional[OllamaEmbeddings] = None
        self.vector_store: Optional[FAISS] = None
        self.splitter: Optional[HierarchicalSplitter] = None
        self.retriever: Optional[ContextAwareRetriever] = None
        self.llm: Optional[ChatOllama] = None
        self.documents: List[Document] = []
        
        # 성능 통계
        self.stats: Dict[str, Union[int, float]] = {
            'total_documents': 0,
            'total_chunks': 0,
            'index_build_time': 0.0,
            'last_query_time': 0.0,
            'queries_processed': 0
        }
        
        print("RAG Pipeline 초기화:")
        print(f"  - 임베딩 모델: {embed_model}")
        print(f"  - LLM 모델: {llm_model}")
        print(f"  - 데이터 파일: {data_file}")
        print(f"  - 벡터 저장소: {vector_store_path}")
    
    def initialize_models(self):
        """모델 초기화"""
        print("모델 초기화 중...")
        
        # 임베딩 모델
        self.embeddings = OllamaEmbeddings(model=self.embed_model)
        
        # LLM 모델 (스트리밍 지원)
        self.llm = ChatOllama(
            model=self.llm_model,
            temperature=0.1,
            streaming=True,
            callbacks=[StreamingStdOutCallbackHandler()]
        )
        
        # 문서 분할기
        self.splitter = HierarchicalSplitter(
            chunk_size=self.chunk_size,
            overlap_ratio=self.overlap_ratio,
            preserve_tables=True,
            preserve_code=True
        )
        
        print("✓ 모델 초기화 완료")
    
    def load_and_process_documents(self, force_rebuild: bool = False):
        """문서 로드 및 처리"""
        print("문서 처리 시작...")
        start_time = time.time()
        
        # 기존 벡터 스토어 확인
        if not force_rebuild and os.path.exists(self.vector_store_path) and self.embeddings is not None:
            try:
                print("기존 벡터 스토어 로드 중...")
                self.vector_store = FAISS.load_local(
                    self.vector_store_path, 
                    self.embeddings, 
                    allow_dangerous_deserialization=True
                )
                
                # 메타데이터 로드
                metadata_file = f"{self.vector_store_path}/metadata.json"
                if os.path.exists(metadata_file):
                    with open(metadata_file, 'r', encoding='utf-8') as f:
                        self.stats.update(json.load(f))
                
                print(f"✓ 기존 벡터 스토어 로드 완료 (문서: {self.stats.get('total_chunks', 0)}개)")
                return
                
            except Exception as e:
                print(f"기존 벡터 스토어 로드 실패: {e}")
                print("새로 생성합니다...")
        
        # 문서 로드
        if not os.path.exists(self.data_file):
            raise FileNotFoundError(f"데이터 파일을 찾을 수 없습니다: {self.data_file}")
        
        with open(self.data_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        print(f"원본 문서 크기: {len(content):,} 문자")
        
        # 계층적 분할
        if self.splitter is None:
            raise ValueError("문서 분할기가 초기화되지 않았습니다. initialize_models()를 먼저 호출하세요.")
        
        print("계층적 문서 분할 중...")
        self.documents = self.splitter.split_document(content)
        
        self.stats['total_documents'] = 1
        self.stats['total_chunks'] = len(self.documents)
        
        print(f"✓ 분할 완료: {len(self.documents)}개 청크")
        
        # 벡터 스토어 구축
        if self.embeddings is None:
            raise ValueError("임베딩 모델이 초기화되지 않았습니다. initialize_models()를 먼저 호출하세요.")
            
        print("벡터 인덱스 구축 중...")
        self.vector_store = FAISS.from_documents(self.documents, self.embeddings)
        
        # 벡터 스토어 저장
        os.makedirs(self.vector_store_path, exist_ok=True)
        self.vector_store.save_local(self.vector_store_path)
        
        # 메타데이터 저장
        self.stats['index_build_time'] = time.time() - start_time
        metadata_file = f"{self.vector_store_path}/metadata.json"
        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(self.stats, f, ensure_ascii=False, indent=2)
        
        print(f"✓ 벡터 인덱스 구축 완료 (소요시간: {self.stats['index_build_time']:.2f}초)")
    
    def build_retriever(self):
        """검색기 구축"""
        print("검색기 구축 중...")
        
        if not self.vector_store:
            raise ValueError("벡터 스토어가 초기화되지 않았습니다. load_and_process_documents()를 먼저 호출하세요.")
        
        # FAISS 검색기
        vector_retriever = self.vector_store.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 15, "fetch_k": 50, "lambda_mult": 0.7}
        )
        
        # BM25 검색기
        if not self.documents:
            # 벡터 스토어에서 문서 복원 (간소화된 버전)
            print("문서를 벡터 스토어에서 복원 중...")
            # 실제 구현에서는 문서 메타데이터를 별도 저장하는 것이 좋습니다
            self.documents = [Document(page_content="임시 문서", metadata={})]
        
        bm25_retriever = BM25Retriever.from_documents(
            self.documents, 
            bm25_params={"k1": 1.5, "b": 0.75}
        )
        bm25_retriever.k = 20
        
        # 앙상블 검색기
        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=[0.4, 0.6]
        )
        
        # 컨텍스트 인식 검색기로 래핑
        self.retriever = ContextAwareRetriever(
            base_retriever=ensemble_retriever,
            rerank_top_k=20,
            final_top_k=self.final_top_k,
            enable_query_expansion=True,
            noise_threshold=0.7
        )
        
        print("✓ 검색기 구축 완료")
    
    def create_prompt_template(self) -> PromptTemplate:
        """프롬프트 템플릿 생성"""
        
        template = """당신은 원스토어 인앱결제 기술문서 전문가입니다. 
주어진 문서를 바탕으로 사용자의 질문에 정확하고 자세하게 답변해주세요.

**답변 지침:**
1. 기술적 정확성을 최우선으로 합니다
2. 코드 예제가 있다면 반드시 포함합니다  
3. API 파라미터나 응답값은 정확히 기술합니다
4. 단계별 설명이 필요한 경우 번호를 매겨 설명합니다
5. 불확실한 내용은 "문서에서 확인되지 않음"이라고 명시합니다

**참고 문서:**
{context}

**질문:** {question}

**답변:**"""

        return PromptTemplate(
            template=template,
            input_variables=["context", "question"]
        )
    
    def query(self, question: str, return_sources: bool = True, stream: bool = True) -> Dict[str, Any]:
        """쿼리 실행"""
        start_time = time.time()
        
        if not self.retriever or not self.llm:
            raise ValueError("파이프라인이 완전히 초기화되지 않았습니다.")
        
        print(f"\n🔍 질문: {question}")
        print("=" * 60)
        
        # 1. 문서 검색
        print("관련 문서 검색 중...")
        retrieved_docs = self.retriever.get_relevant_documents(question)
        
        print(f"✓ {len(retrieved_docs)}개 문서 검색됨")
        
        # 2. 컨텍스트 구성
        context = "\n\n".join([
            f"[문서 {i+1}] {doc.metadata.get('section_hierarchy', 'N/A')}\n{doc.page_content}"
            for i, doc in enumerate(retrieved_docs)
        ])
        
        # 3. 프롬프트 생성
        prompt_template = self.create_prompt_template()
        prompt = prompt_template.format(context=context, question=question)
        
        # 4. LLM 실행
        print("\n💭 답변 생성 중...")
        print("-" * 60)
        
        response_text = ""
        if stream:
            # 스트리밍 응답
            response_generator = self.llm.stream(prompt)
            for chunk in response_generator:
                if hasattr(chunk, 'content'):
                    # chunk.content는 다양한 타입이 될 수 있으므로 str로 안전하게 변환
                    response_text += str(chunk.content)
            print()  # 줄바꿈
        else:
            # 일반 응답
            response = self.llm.invoke(prompt)
            if hasattr(response, 'content'):
                # response.content도 str로 안전하게 변환
                response_text = str(response.content)
            else:
                response_text = str(response)
        
        # 5. 통계 업데이트
        query_time = time.time() - start_time
        self.stats['last_query_time'] = query_time
        self.stats['queries_processed'] += 1
        
        print("-" * 60)
        print(f"⏱️  응답 시간: {query_time:.2f}초")
        
        # 6. 결과 구성
        result = {
            'question': question,
            'answer': response_text,
            'query_time': query_time,
            'retrieved_docs_count': len(retrieved_docs)
        }
        
        if return_sources:
            result['sources'] = [
                {
                    'content': doc.page_content[:200] + "...",
                    'metadata': doc.metadata
                }
                for doc in retrieved_docs
            ]
        
        return result
    
    def batch_query(self, questions: List[str]) -> List[Dict[str, Any]]:
        """배치 쿼리 실행"""
        print(f"📋 배치 쿼리 실행: {len(questions)}개 질문")
        
        results = []
        for i, question in enumerate(questions, 1):
            print(f"\n[{i}/{len(questions)}]")
            result = self.query(question, stream=False)
            results.append(result)
        
        return results
    
    def get_statistics(self) -> Dict[str, Any]:
        """파이프라인 통계 반환"""
        return {
            **self.stats,
            'models': {
                'embedding': self.embed_model,
                'llm': self.llm_model
            },
            'config': {
                'chunk_size': self.chunk_size,
                'overlap_ratio': self.overlap_ratio,
                'final_top_k': self.final_top_k
            }
        }
    
    def analyze_document_structure(self) -> Dict[str, Any]:
        """문서 구조 분석"""
        if not self.documents:
            return {"error": "문서가 로드되지 않았습니다"}
        
        # 출처별 통계
        source_stats: Dict[str, int] = {}
        content_type_stats: Dict[str, int] = {}
        tech_term_stats: Dict[str, Dict[str, int]] = {}
        
        for doc in self.documents:
            # 출처 통계
            source_url = doc.metadata.get('source_url', 'Unknown')
            source_domain = source_url.split('/')[-1] if '/' in source_url else source_url
            source_stats[source_domain] = source_stats.get(source_domain, 0) + 1
            
            # 콘텐츠 타입 통계
            content_types = doc.metadata.get('content_types', ['text'])
            for ct in content_types:
                content_type_stats[ct] = content_type_stats.get(ct, 0) + 1
            
            # 기술 용어 통계
            tech_terms = doc.metadata.get('technical_terms', {})
            for category, terms in tech_terms.items():
                if category not in tech_term_stats:
                    tech_term_stats[category] = {}
                for term in terms:
                    tech_term_stats[category][term] = tech_term_stats[category].get(term, 0) + 1
        
        return {
            'total_chunks': len(self.documents),
            'source_distribution': source_stats,
            'content_type_distribution': content_type_stats,
            'tech_term_distribution': {
                category: dict(list(sorted(terms.items(), key=lambda x: x[1], reverse=True))[:10])
                for category, terms in tech_term_stats.items()
            }
        }
    
    def debug_search(self, question: str) -> Dict[str, Any]:
        """검색 디버깅 정보"""
        if not self.retriever:
            return {"error": "검색기가 초기화되지 않았습니다"}
        
        # 쿼리 분석
        analytics = self.retriever.get_search_analytics(question)
        
        # 검색 실행
        retrieved_docs = self.retriever.get_relevant_documents(question)
        
        # 검색 결과 분석
        result_analysis = []
        for i, doc in enumerate(retrieved_docs):
            result_analysis.append({
                'rank': i + 1,
                'content_preview': doc.page_content[:150] + "...",
                'section_hierarchy': doc.metadata.get('section_hierarchy', 'N/A'),
                'content_types': doc.metadata.get('content_types', []),
                'technical_terms': doc.metadata.get('technical_terms', {}),
                'content_length': len(doc.page_content)
            })
        
        return {
            'query_analysis': analytics,
            'retrieved_count': len(retrieved_docs),
            'results': result_analysis
        }


# 편의 함수들
def create_pipeline(data_file: str = "data/dev_center_guide_allmd_touched.md",
                   force_rebuild: bool = False) -> OptimizedRAGPipeline:
    """파이프라인 생성 및 초기화"""
    
    pipeline = OptimizedRAGPipeline(data_file=data_file)
    
    # 초기화 단계
    pipeline.initialize_models()
    pipeline.load_and_process_documents(force_rebuild=force_rebuild)
    pipeline.build_retriever()
    
    print("\n🚀 RAG 파이프라인 준비 완료!")
    print(f"   총 문서: {pipeline.stats['total_chunks']}개")
    print(f"   인덱싱 시간: {pipeline.stats['index_build_time']:.2f}초")
    
    return pipeline


def interactive_mode(pipeline: OptimizedRAGPipeline):
    """대화형 모드"""
    print("\n" + "="*60)
    print("🤖 원스토어 IAP 기술문서 QA 시스템")
    print("="*60)
    print("질문을 입력하세요 (종료: 'quit' 또는 'exit')")
    print("특수 명령어:")
    print("  - 'stats': 파이프라인 통계")
    print("  - 'analyze': 문서 구조 분석")
    print("  - 'debug <질문>': 검색 디버깅")
    print("-"*60)
    
    while True:
        try:
            question = input("\n❓ 질문: ").strip()
            
            if question.lower() in ['quit', 'exit', '종료']:
                print("👋 시스템을 종료합니다.")
                break
            
            if question == 'stats':
                stats = pipeline.get_statistics()
                print("\n📊 파이프라인 통계:")
                print(json.dumps(stats, ensure_ascii=False, indent=2))
                continue
            
            if question == 'analyze':
                analysis = pipeline.analyze_document_structure()
                print("\n📈 문서 구조 분석:")
                print(json.dumps(analysis, ensure_ascii=False, indent=2))
                continue
            
            if question.startswith('debug '):
                debug_query = question[6:]
                debug_info = pipeline.debug_search(debug_query)
                print("\n🔍 검색 디버깅:")
                print(json.dumps(debug_info, ensure_ascii=False, indent=2))
                continue
            
            if not question:
                continue
            
            # 일반 질문 처리
            pipeline.query(question)
            
        except KeyboardInterrupt:
            print("\n\n👋 시스템을 종료합니다.")
            break
        except Exception as e:
            print(f"\n❌ 오류 발생: {e}")


# 테스트 함수
def test_pipeline():
    """파이프라인 테스트"""
    
    test_questions = [
        "PurchaseClient 초기화 방법이 뭔가요?",
        "PNS의 purchaseState 값은 무엇인가요?", 
        "PNS 서비스 설정 방법을 알려주세요",
        "인앱결제 에러가 발생했을 때 해결 방법은?",
        "구독형 상품과 관리형 상품의 차이점은?"
    ]
    
    print("🧪 파이프라인 테스트 시작...")
    
    # 테스트용 더미 파이프라인 (실제 파일 없이)
    # 실제 사용시에는 create_pipeline()을 사용
    
    for question in test_questions:
        print(f"\n질문: {question}")
        # result = pipeline.query(question)
        print("답변: [테스트 모드 - 실제 구현에서는 답변이 생성됩니다]")
        print("-" * 60)


# if __name__ == "__main__":
#     # 테스트 실행
#     test_pipeline()


In [52]:
def demo_quick_test():
    """빠른 테스트 데모"""
    print("🚀 빠른 테스트 데모 시작")
    print("=" * 60)
    
    # 설정
    data_file = "data/dev_center_guide_allmd_touched.md"
    
    # 데이터 파일 존재 확인
    if not os.path.exists(data_file):
        print(f"❌ 데이터 파일을 찾을 수 없습니다: {data_file}")
        print("경로를 수정하거나 파일을 확인해주세요.")
        return
    
    try:
        # 파이프라인 생성
        print("파이프라인 초기화 중...")
        pipeline = OptimizedRAGPipeline(
            data_file=data_file,
            chunk_size=800,
            final_top_k=3
        )
        
        # 단계별 초기화
        pipeline.initialize_models()
        pipeline.load_and_process_documents(force_rebuild=False)
        pipeline.build_retriever()
        
        print("\n✅ 파이프라인 준비 완료!")
        
        # 테스트 질문들
        test_questions = [
            "PurchaseClient를 어떻게 초기화하나요?",
            "PNS의 purchaseState 값의 종류는 무엇인가요?",
            "PNS 서비스란 무엇인가요?"
        ]
        
        print("\n📝 테스트 질문 실행:")
        print("-" * 40)
        
        for i, question in enumerate(test_questions, 1):
            print(f"\n[질문 {i}] {question}")
            print("🤔 답변 생성 중...")
            
            start_time = time.time()
            result = pipeline.query(question, stream=False)
            elapsed = time.time() - start_time
            
            print("\n💡 답변:")
            print(result['answer'])
            print(f"\n⏱️  응답 시간: {elapsed:.2f}초")
            print("-" * 40)
        
        # 통계 출력
        stats = pipeline.get_statistics()
        print("\n📊 파이프라인 통계:")
        print(f"  - 총 문서 청크: {stats['total_chunks']}")
        print(f"  - 인덱싱 시간: {stats['index_build_time']:.2f}초")
        print(f"  - 처리된 쿼리: {stats['queries_processed']}")
        print(f"  - 마지막 쿼리 시간: {stats['last_query_time']:.2f}초")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        
demo_quick_test()


🚀 빠른 테스트 데모 시작
파이프라인 초기화 중...
RAG Pipeline 초기화:
  - 임베딩 모델: bge-m3:latest
  - LLM 모델: exaone3.5:latest
  - 데이터 파일: data/dev_center_guide_allmd_touched.md
  - 벡터 저장소: models/faiss_optimized
모델 초기화 중...
✓ 모델 초기화 완료
문서 처리 시작...
기존 벡터 스토어 로드 중...
✓ 기존 벡터 스토어 로드 완료 (문서: 5912개)
검색기 구축 중...
문서를 벡터 스토어에서 복원 중...
❌ 오류 발생: "ContextAwareRetriever" object has no field "base_retriever"


Traceback (most recent call last):
  File "/tmp/ipykernel_5434/1986687736.py", line 27, in demo_quick_test
    pipeline.build_retriever()
  File "/tmp/ipykernel_5434/2927573387.py", line 206, in build_retriever
    self.retriever = ContextAwareRetriever(
                     ^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_5434/4081965214.py", line 298, in __init__
    self.base_retriever = base_retriever
    ^^^^^^^^^^^^^^^^^^^
  File "/home/gpu/dev/jupyter_root/venv/lib/python3.11/site-packages/pydantic/main.py", line 884, in __setattr__
    raise ValueError(f'"{self.__class__.__name__}" object has no field "{name}"')
ValueError: "ContextAwareRetriever" object has no field "base_retriever"


In [53]:
"""
수정된 ContextAwareRetriever 클래스
Pydantic 우회 방식으로 해결
"""

from typing import List, Dict, Any, Optional, Union
from langchain.schema import BaseRetriever
from langchain.docstore.document import Document
from langchain_core.callbacks.manager import CallbackManagerForRetrieverRun


class FixedContextAwareRetriever(BaseRetriever):
    """컨텍스트 인식 검색기 - 간단한 버전"""
    
    def __init__(self, 
                 base_retriever: BaseRetriever,
                 rerank_top_k: int = 20,
                 final_top_k: int = 5,
                 enable_query_expansion: bool = True,
                 noise_threshold: float = 0.7,
                 **kwargs):
        
        # 기본 초기화 (kwargs는 부모 클래스에 전달)
        super().__init__(**kwargs)
        
        # 속성 직접 설정 (Pydantic 검증 우회)
        object.__setattr__(self, 'base_retriever', base_retriever)
        object.__setattr__(self, 'rerank_top_k', rerank_top_k)
        object.__setattr__(self, 'final_top_k', final_top_k)
        object.__setattr__(self, 'enable_query_expansion', enable_query_expansion)
        object.__setattr__(self, 'noise_threshold', noise_threshold)
        
        # 내부 객체 초기화
        object.__setattr__(self, 'query_analyzer', QueryAnalyzer())
        object.__setattr__(self, 'context_scorer', ContextScorer())
        object.__setattr__(self, 'noise_filter', NoiseFilter())
    
    def _get_relevant_documents(self, 
                              query: str, 
                              *, 
                              run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        """메인 검색 메소드"""
        
        # 1. 쿼리 분석
        search_context = self.query_analyzer.analyze_query(query)
        
        # 2. 쿼리 확장 (옵션)
        queries_to_search = [query]
        if self.enable_query_expansion:
            expanded_queries = self.query_analyzer.expand_query(query)
            queries_to_search.extend(expanded_queries[:3])  # 최대 3개 확장
        
        # 3. 기본 검색 수행
        all_docs = []
        for q in queries_to_search:
            docs = self.base_retriever.get_relevant_documents(q, run_manager=run_manager)
            all_docs.extend(docs)
        
        # 중복 제거 (page_content 기준)
        unique_docs = []
        seen_contents = set()
        for doc in all_docs:
            content_hash = hash(doc.page_content)
            if content_hash not in seen_contents:
                unique_docs.append(doc)
                seen_contents.add(content_hash)
        
        # 상위 K개만 리랭킹 대상으로
        docs_to_rerank = unique_docs[:self.rerank_top_k]
        
        # 4. 다단계 리랭킹
        reranked_docs = self._rerank_documents(docs_to_rerank, search_context)
        
        # 5. 최종 결과 반환
        return reranked_docs[:self.final_top_k]
    
    def _rerank_documents(self, docs: List[Document], search_context: SearchContext) -> List[Document]:
        """다단계 리랭킹"""
        
        scored_docs = []
        
        for doc in docs:
            # 점수 계산
            term_score = self.context_scorer.calculate_term_relevance(doc, search_context)
            context_score = self.context_scorer.calculate_context_relevance(doc, search_context)
            accuracy_score = self.context_scorer.calculate_technical_accuracy(doc, search_context)
            noise_score = self.noise_filter.calculate_noise_score(doc, search_context)
            
            # 종합 점수 (노이즈는 패널티)
            final_score = (term_score * 0.4 + 
                          context_score * 0.3 + 
                          accuracy_score * 0.2 + 
                          (1 - noise_score) * 0.1)
            
            scored_docs.append((doc, final_score))
        
        # 점수순 정렬
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        # 노이즈 임계값으로 필터링
        filtered_docs = []
        for doc, score in scored_docs:
            noise_score = self.noise_filter.calculate_noise_score(doc, search_context)
            if noise_score <= self.noise_threshold:
                filtered_docs.append(doc)
        
        return filtered_docs if filtered_docs else [doc for doc, _ in scored_docs]
    
    def get_search_analytics(self, query: str) -> Dict[str, Any]:
        """검색 분석 정보 반환 (디버깅용)"""
        search_context = self.query_analyzer.analyze_query(query)
        
        return {
            'query_terms': search_context.query_terms,
            'query_type': search_context.query_type,
            'tech_terms': search_context.tech_terms,
            'context_keywords': list(search_context.context_keywords)
        }

print("✓ 수정된 ContextAwareRetriever 클래스 정의 완료")


✓ 수정된 ContextAwareRetriever 클래스 정의 완료


In [54]:
"""
수정된 OptimizedRAGPipeline 클래스
FixedContextAwareRetriever 사용
"""

# OptimizedRAGPipeline 클래스의 build_retriever 메소드만 수정
class FixedOptimizedRAGPipeline(OptimizedRAGPipeline):
    """수정된 RAG 파이프라인 - 고정된 검색기 사용"""
    
    def build_retriever(self):
        """검색기 구축 - 수정된 버전"""
        print("검색기 구축 중...")
        
        if not self.vector_store:
            raise ValueError("벡터 스토어가 초기화되지 않았습니다. load_and_process_documents()를 먼저 호출하세요.")
        
        # FAISS 검색기
        vector_retriever = self.vector_store.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 15, "fetch_k": 50, "lambda_mult": 0.7}
        )
        
        # BM25 검색기
        if not self.documents:
            # 벡터 스토어에서 문서 복원 (간소화된 버전)
            print("문서를 벡터 스토어에서 복원 중...")
            # 실제 구현에서는 문서 메타데이터를 별도 저장하는 것이 좋습니다
            self.documents = [Document(page_content="임시 문서", metadata={})]
        
        bm25_retriever = BM25Retriever.from_documents(
            self.documents, 
            bm25_params={"k1": 1.5, "b": 0.75}
        )
        bm25_retriever.k = 20
        
        # 앙상블 검색기
        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=[0.4, 0.6]
        )
        
        # 컨텍스트 인식 검색기로 래핑 - 수정된 클래스 사용
        self.retriever = FixedContextAwareRetriever(
            base_retriever=ensemble_retriever,
            rerank_top_k=20,
            final_top_k=self.final_top_k,
            enable_query_expansion=True,
            noise_threshold=0.7
        )
        
        print("✓ 검색기 구축 완료")

# 편의 함수도 수정
def create_fixed_pipeline(data_file: str = "data/dev_center_guide_allmd_touched.md",
                         force_rebuild: bool = False) -> FixedOptimizedRAGPipeline:
    """수정된 파이프라인 생성 및 초기화"""
    
    pipeline = FixedOptimizedRAGPipeline(data_file=data_file)
    
    # 초기화 단계
    pipeline.initialize_models()
    pipeline.load_and_process_documents(force_rebuild=force_rebuild)
    pipeline.build_retriever()
    
    print("\n🚀 RAG 파이프라인 준비 완료!")
    print(f"   총 문서: {pipeline.stats['total_chunks']}개")
    print(f"   인덱싱 시간: {pipeline.stats['index_build_time']:.2f}초")
    
    return pipeline

print("✓ 수정된 OptimizedRAGPipeline 클래스 정의 완료")


✓ 수정된 OptimizedRAGPipeline 클래스 정의 완료


In [59]:
"""
수정된 테스트 함수
"""

def demo_fixed_test():
    """수정된 빠른 테스트 데모"""
    print("🚀 수정된 테스트 데모 시작")
    print("=" * 60)
    
    # 설정
    data_file = "data/dev_center_guide_allmd_touched.md"
    
    # 데이터 파일 존재 확인
    if not os.path.exists(data_file):
        print(f"❌ 데이터 파일을 찾을 수 없습니다: {data_file}")
        print("경로를 수정하거나 파일을 확인해주세요.")
        return
    
    try:
        # 수정된 파이프라인 생성
        print("파이프라인 초기화 중...")
        pipeline = create_fixed_pipeline(
            data_file=data_file,
            force_rebuild=False  # 기존 인덱스 재사용
        )
        
        print("\n✅ 파이프라인 준비 완료!")
        
        # 테스트 질문들
        test_questions = [
            "PurchaseClient를 어떻게 초기화하나요?",
            "PNS의 purchaseState 값의 종류는 무엇인가요?",
            "PNS 서비스란 무엇인가요?"
        ]
        
        print("\n📝 테스트 질문 실행:")
        print("-" * 40)
        
        for i, question in enumerate(test_questions, 1):
            print(f"\n[질문 {i}] {question}")
            print("🤔 답변 생성 중...")
            
            start_time = time.time()
            result = pipeline.query(question, stream=False)
            elapsed = time.time() - start_time
            
            print("\n💡 답변:")
            print(result['answer'])
            print(f"\n⏱️  응답 시간: {elapsed:.2f}초")
            print("\n\n")
            print("*" * 200)
        
        # 통계 출력
        stats = pipeline.get_statistics()
        print("\n📊 파이프라인 통계:")
        print(f"  - 총 문서 청크: {stats['total_chunks']}")
        print(f"  - 인덱싱 시간: {stats['index_build_time']:.2f}초")
        print(f"  - 처리된 쿼리: {stats['queries_processed']}")
        print(f"  - 마지막 쿼리 시간: {stats['last_query_time']:.2f}초")
        
        return pipeline
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

# 실행 - 다시 시도
print("🔄 수정된 클래스로 다시 테스트...")
pipeline = demo_fixed_test()


🔄 수정된 클래스로 다시 테스트...
🚀 수정된 테스트 데모 시작
파이프라인 초기화 중...
RAG Pipeline 초기화:
  - 임베딩 모델: bge-m3:latest
  - LLM 모델: exaone3.5:latest
  - 데이터 파일: data/dev_center_guide_allmd_touched.md
  - 벡터 저장소: models/faiss_optimized
모델 초기화 중...
✓ 모델 초기화 완료
문서 처리 시작...
기존 벡터 스토어 로드 중...
✓ 기존 벡터 스토어 로드 완료 (문서: 5912개)
검색기 구축 중...
문서를 벡터 스토어에서 복원 중...
✓ 검색기 구축 완료

🚀 RAG 파이프라인 준비 완료!
   총 문서: 5912개
   인덱싱 시간: 73.25초

✅ 파이프라인 준비 완료!

📝 테스트 질문 실행:
----------------------------------------

[질문 1] PurchaseClient를 어떻게 초기화하나요?
🤔 답변 생성 중...

🔍 질문: PurchaseClient를 어떻게 초기화하나요?
관련 문서 검색 중...
✓ 5개 문서 검색됨

💭 답변 생성 중...
------------------------------------------------------------
PurchaseClient를 초기화하는 방법은 다음과 같습니다. 주로 Kotlin과 Java 언어를 사용하여 구현하는 방법을 설명드리겠습니다. 주어진 문서를 바탕으로 단계별로 설명드리겠습니다.

### Kotlin 구현 예시

1. **PurchasesUpdatedListener 구현**: 구매 결과를 처리할 리스너를 먼저 정의합니다.
2. **PurchaseClient Builder 사용**: `PurchaseClient.newBuilder()`를 사용하여 PurchaseClient 인스턴스를 생성하고 필요한 설정을 적용합니다.

```kotlin
// 1. 구매 결과 리스너 정의
private val listener 